# Core

> Core module, with the main `Class` `ConsultaFipe` implementation.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import json
import requests
import pandas as pd

class ConsultaFipe:
    def __init__(self, 
                 mes: str = 'agosto/2022', 
                 tipo_veiculo: str = 'carro',
                 marca: str = 'VW - VolksWagen',
                 modelo: str = 'T-Cross Sense 1.0 TSI Flex 5p Aut.',
                 ano_modelo: str = 2021,
                 combustivel: str = 'Gasolina'):
        self._base_url = "http://veiculos.fipe.org.br/api/veiculos/"

        self.mes = mes
        self.tabela_referencia = self.consulta_tabela_referencia()
        self.codigo_tabela_referencia = int(self.tabela_referencia.query(f"mes == '{self.mes}'").loc[:,'codigo_mes'].values[0])

        self.tipo_veiculo = tipo_veiculo
        self.tipos_veiculos = {'carro': 1, 'moto': 2, 'caminhao': 3}
        self.codigo_tipo_veiculo = self.tipos_veiculos[self.tipo_veiculo]

        self.marca = marca
        self.tabela_marcas = self.consulta_tabela_marcas()
        self.codigo_marca = int(self.tabela_marcas[self.tabela_marcas['marca'].str.contains(self.marca)].loc[:,'codigo_marca'].values[0])

        self.modelo = modelo
        self.tabela_modelos = self.consulta_tabela_modelos()
        self.codigo_modelo = int(self.tabela_modelos.query(f"modelo == '{self.modelo}'").loc[:,'codigo_modelo'].values[0])
        
        self.ano_modelo = ano_modelo
        self.combustivel = combustivel
        self.codigo_tipo_combustivel = 1 if self.combustivel == 'Gasolina' else 2

    def preco(self):
        url = f"{self._base_url}/ConsultarValorComTodosParametros"
        params = {'codigoTabelaReferencia': self.codigo_tabela_referencia,
                  'codigoTipoVeiculo': self.codigo_tipo_veiculo,
                  'codigoMarca': self.codigo_marca,
                  'codigoModelo': self.codigo_modelo,
                  'anoModelo': self.ano_modelo,
                  'codigoTipoCombustivel': self.codigo_tipo_combustivel,
                  'tipoConsulta': 'tradicional'}
        req = requests.post(url, data=params)
        req_text = req.text
        req_json = json.loads(req_text)
        return pd.DataFrame([req_json])

    def consulta_tabela_referencia(self):
        url = f"{self._base_url}/ConsultarTabelaDeReferencia"
        req = requests.post(url)
        req_text = req.text
        req_json = json.loads(req_text)
        df = pd.DataFrame(req_json)
        df.columns = ['codigo_mes', 'mes']
        df['mes'] = df['mes'].str.strip()

        return df

    def consulta_tabela_marcas(self):
        url = f"{self._base_url}/ConsultarMarcas"
        body = {"codigoTabelaReferencia": self.codigo_tabela_referencia, "codigoTipoVeiculo": self.codigo_tipo_veiculo}
        req = requests.post(url, data=body)
        req_text = req.text
        req_json = json.loads(req_text)
        df = pd.DataFrame(req_json).rename(columns={'Label': 'marca', 'Value': 'codigo_marca'})
        df['mes'] = self.mes
        return df

    def consulta_tabela_modelos(self):
        url = f"{self._base_url}/ConsultarModelos"
        body = {"codigoTabelaReferencia": self.codigo_tabela_referencia,
                "codigoTipoVeiculo": self.codigo_tipo_veiculo,
                "codigoMarca": self.codigo_marca}
        req = requests.post(url, data=body)
        req_text = req.text
        req_json = json.loads(req_text)
        df = pd.DataFrame(req_json['Modelos']).rename(columns={'Label': 'modelo', 'Value': 'codigo_modelo'})
        return df

    def consulta_tabela_ano_modelo(self):
        url = f"{self._base_url}/ConsultarAnoModelo"
        body = {"codigoTabelaReferencia": self.codigo_tabela_referencia,
                "codigoTipoVeiculo": self.codigo_tipo_veiculo,
                "codigoMarca": self.codigo_marca,
                "codigoModelo": self.codigo_modelo}
        req = requests.post(url, data=body)
        req_text = req.text
        req_json = json.loads(req_text)
        df = pd.DataFrame(req_json).rename(columns={'Label': 'Ano_Modelo', 'Value': 'codigo'})

        # Substituir "32000 Gasolina" por "zero km"
        df['Ano_Modelo'] = df['Ano_Modelo'].replace('32000 Gasolina', 'Zero Km Gasolina')
        return df

    def update_codigo_tabela_referencia(self):
        self.codigo_tabela_referencia = int(self.tabela_referencia.query(f"mes == '{self.mes}'").loc[:,'codigo_mes'].values[0])

In [ ]:
fipe = ConsultaFipe()

In [ ]:
fipe.preco()

,Valor,Marca,Modelo,AnoModelo,Combustivel,CodigoFipe,MesReferencia,Autenticacao,TipoVeiculo,SiglaCombustivel,DataConsulta
0,"R$ 101.120,00",VW - VolksWagen,T-Cross Sense 1.0 TSI Flex 5p Aut.,2021,Gasolina,005520-4,agosto de 2022,f4wq6m4pj4djf,1,G,"quarta-feira, 24 de agosto de 2022 09:55"


Caso não saiba qual o código da marca do seu carro, poderá fazer uma busca na tabela de marcas:

In [ ]:
fipe.tabela_marcas.head()

,marca,codigo_marca,mes
0,Acura,1,agosto/2022
1,Agrale,2,agosto/2022
2,Alfa Romeo,3,agosto/2022
3,AM Gen,4,agosto/2022
4,Asia Motors,5,agosto/2022


In [ ]:
fipe.tabela_marcas[fipe.tabela_marcas['marca'].str.contains('vw', case=False)]

,marca,codigo_marca,mes
89,VW - VolksWagen,59,agosto/2022


E caso também não saiba o nome padrão fipe do modelo do seu carro, basta fazer uma busca na tabela de modelos:

In [ ]:
fipe.tabela_modelos.head()

,modelo,codigo_modelo
0,AMAROK CD2.0 16V/S CD2.0 16V TDI 4x2 Die,5585
1,AMAROK CD2.0 16V/S CD2.0 16V TDI 4x4 Die,5586
2,AMAROK Comfor. 3.0 V6 TDI 4x4 Dies. Aut.,9895
3,AMAROK Comfor. CD 2.0 TDI 4x4 Dies. Aut.,8531
4,AMAROK CS2.0 16V/S2.0 16V TDI 4x2 Diesel,5748


In [ ]:
fipe.tabela_modelos[fipe.tabela_modelos['modelo'].str.contains('t-cross', case=False)]

,modelo,codigo_modelo
450,T-Cross 1.0 TSI Flex 12V 5p Aut.,8641
451,T-Cross 1.0 TSI Flex 12V 5p Mec.,8642
452,T-Cross Comfortline 1.0 TSI Flex 5p Aut.,8643
453,T-Cross Highline 1.4 TSI Flex 16V 5p Aut,8644
454,T-Cross Sense 1.0 TSI Flex 5p Aut.,9564
